# PDF processing as images

Посмотрев на данные и увидев, что уже на 20 первых страницах изображений встречается 37 штук, а ещё и из-за того, что это презенташка, текст нормально не собирается - не понятно, какая цифра к чему относится, я решила сделать предобработку данных чуть иначе - использовать мульмодальную llm для всех слайдов

In [1]:
from pdf2image import convert_from_path
import os

def save_pdf_pages_as_images(pdf_path, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Конвертируем страницы PDF в изображения
    images = convert_from_path(pdf_path)  
    
    # Сохраняем каждую страницу как изображение
    for i, image in enumerate(images):
        image_path = os.path.join(output_dir, f'page_{i + 1}.png')
        image.save(image_path, 'PNG')
        print(f'Сохранена страница {i + 1} как {image_path}')

# Пример использования функции
pdf_path = '../data/raw/Сбер 2023-1-20.pdf'
output_dir = '../data/interim/images/'
save_pdf_pages_as_images(pdf_path, output_dir)

Сохранена страница 1 как ../data/interim/images/page_1.png
Сохранена страница 2 как ../data/interim/images/page_2.png
Сохранена страница 3 как ../data/interim/images/page_3.png
Сохранена страница 4 как ../data/interim/images/page_4.png
Сохранена страница 5 как ../data/interim/images/page_5.png
Сохранена страница 6 как ../data/interim/images/page_6.png
Сохранена страница 7 как ../data/interim/images/page_7.png
Сохранена страница 8 как ../data/interim/images/page_8.png
Сохранена страница 9 как ../data/interim/images/page_9.png
Сохранена страница 10 как ../data/interim/images/page_10.png
Сохранена страница 11 как ../data/interim/images/page_11.png
Сохранена страница 12 как ../data/interim/images/page_12.png
Сохранена страница 13 как ../data/interim/images/page_13.png
Сохранена страница 14 как ../data/interim/images/page_14.png
Сохранена страница 15 как ../data/interim/images/page_15.png
Сохранена страница 16 как ../data/interim/images/page_16.png
Сохранена страница 17 как ../data/interim/

In [6]:
import base64
import os

from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


# Функция кодирования изображения в формат base64
def encode_image(image_path):
    """
    Функция для кодирования изображения в формат base64.

    Аргументы:
    image_path: Строка, путь к изображению, которое нужно закодировать.

    Возвращает:
    Закодированное в формате base64 изображение в виде строки.
    """
    with open(image_path, "rb") as image_file:
        # Читаем файл изображения в бинарном режиме и кодируем в base64
        return base64.b64encode(image_file.read()).decode("utf-8")


# Функция для суммаризации изображения с использованием модели GPT
def image_summarize(img_base64, prompt):
    """
    Функция для получения суммаризации изображения с использованием GPT модели.

    Аргументы:
    img_base64: Строка, изображение закодированное в формате base64.
    prompt: Строка, запрос для модели GPT, содержащий инструкцию для суммаризации изображения.

    Возвращает:
    Суммаризация изображения, возвращенная моделью GPT.
    """
    chat = ChatOpenAI(model="gpt-4o-mini")
    msg = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},  # Запрос для модели
                    {
                        "type": "image_url",  # Тип содержимого - изображение
                        "image_url": {"url": f"data:image/png;base64,{img_base64}"},  # Изображение в формате base64
                    },
                ]
            )
        ]
    )
    # Возвращаем содержимое ответа от модели
    return msg.content

In [8]:
def prepare_text_from_image(img_path):
    prompt = """Опиши ключевую информацию, которая представлена на изображении. Описание должно быть конкретным и точным. Обрати особое внимание на графики, диаграммы или визуальные элементы, которые можно проанализировать.
    Очень важно не упустить детали. Ответь в формате Markdown."""
    base64_image = encode_image(img_path)
    return image_summarize(base64_image, prompt)

In [10]:
import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass.getpass("openai: ")

openai:  ········


In [12]:
res = prepare_text_from_image("../data/interim/images/page_11.png")

In [13]:
print(res)

# Ключевая информация о Сбер

## Общие данные
- **Розничные клиенты**: 
  - 108,5 млн активных
  - 81,9 млн MAU Сбербанк Онлайн
  - 10,0 млн пользователей подписки СберПрайм
- **Корпоративные клиенты**: 
  - 3,2 млн активных
  - 2,8 млн MAU СберБизнес Онлайн
- **Сотрудники**: 210,8 тыс.
- **Офисы**: ~12,0 тыс.

## Важные факты
- **Сильнейший финансовый бренд в Европе** (по рейтингу Brand Finance)
- **№ 1 по рыночной капитализации в России**: ~75 млрд долларов США
- **Крупнейший эквайер в Европе** (по Nilson Report)
- **Банк № 1** по чистой прибыли в Восточной Европе по итогам 2023 года

## Трансформация Сбера
1. **Надежный банк**: 
   - Лидер в депозитах физических лиц
   - Крупнейшая сеть отделений среди банков в России
   - Крупнейший транзакционный банк страны
2. **Лидер в цифровом банкинге**:
   - "Лучшее банковское приложение"
   - Лучшие практики управления рисками
   - Лучший клиентский опыт
3. **Интегрированная технологическая экосистема**:
   - Построение экосистем вокруг клие

In [14]:
res2 = prepare_text_from_image("../data/interim/images/page_18.png")
print(res2)

# Искусственный интеллект в 2023 году

## Результаты AI-трансформации
- **Финансовый эффект от внедрения ИИ в 2023 году**: 350+ млрд руб.
- **Среднегодовой рост эффекта от ИИ (CAGR)** за 2020-2023 годы: 53%.
- **Исследователи данных, аналитики данных, дата-инженеры и ML-инженеры**: 2,7 тыс. сотрудников.
- **Модели ИИ, внедренные в бизнес-процессы банка**: 2 тыс. моделей.

## Модели генеративного ИИ
- **GigaChat**: сервис для взаимодействия с пользователями в формате диалога.
  - **Количество пользователей**: 2,6 млн.
- **Kandinsky**: модель для генерации высококачественных изображений по текстовому описанию.
  - **Количество генераций**: 200+ млн изображений.
- **Kandinsky Video**: первая в России модель для создания видео по текстовому описанию.
- **OmniFusion**: мультимодальная модель для описания изображений (вместе с AIRI).

## AI Journey
- **Конференция AI Journey**: важное событие в мире ИИ, прошедшее в 2023 году.
  - **Просмотры трансляции**: 150 млн.
  - **Выступившие спикеры**

In [15]:
res3 = prepare_text_from_image("../data/interim/images/page_20.png")
print(res3)

# Ключевая финансовая информация

## Общие показатели
- **Чистая прибыль**: 1 509 млрд руб. (в 2022 году: 288 млрд руб.)
- **Рентабельность капитала (ROE)**: 25,3% (в 2022 году: 5,2%)
- **Прибыль на акцию (EPS)**: 69,1 руб. (в 2022 году: 13,2 руб.)
- **Собственные средства**: 6,6 трлн руб. (в 2022 году: 5,8 трлн руб.)

## Доходы
- **Чистый процентный доход**: 2 565 млрд руб. (рост на 36,8% по сравнению с предыдущим периодом)
- **Чистый комиссионный доход**: 764 млрд руб. (рост на 9,4% по сравнению с предыдущим периодом)

## Расходы
- **Операционные расходы**: 924 млрд руб. (рост на 27,2% по сравнению с предыдущим периодом)
- **Расходы на резервы**: 265 млрд руб.

## Дополнительные показатели
- **Чистая процентная маржа**: 5,98%
- **Чистые комиссионные доходы к операционным доходам до вычета резервов**: 24,1%
- **Расходы к доходам (CIR)**: 29,2%
- **Стоимость риска (COR)**: 0,8 п.п.

## Кредиты
- **Корпоративные кредиты**: 23,3 трлн руб. (рост на 25,2% по сравнению с 2022 годом)
- **Роз

In [16]:
res4 = prepare_text_from_image("../data/interim/images/page_2.png")
print(res4)

# Ключевая информация об отчете Сбербанка за 2023 год

## Общая информация
- **Название документа**: Годовой отчет ПАО «Сбербанк России» за 2023 год.
- **Период отчета**: с 1 января по 31 декабря 2023 года.
- **Содержание**: Отчет включает данные о результатах деятельности Сбербанка и его ESG-отчет.

## Подход к отчету
- **Законодательство**: Подготовлен в соответствии с законодательством Российской Федерации, включая:
  - Федеральный закон от 26.12.1995 № 208-ФЗ «О акционерных обществах».
  - Федеральный закон от 22.04.1996 № 39-ФЗ «О рынке ценных бумаг».
  - Положение Банка России от 27.03.2020 № 714-П «О раскрытии информации эмитентами ценных бумаг».

## Рекомендации и стандарты
- **Применяемые принципы**:
  - Принципы Глобального договора ООН.
  - Цели устойчивого развития, принятые Генеральной Ассамблеей ООН в 2015 году.
  - Принципы ответственной банковской деятельности (UNEP Finance Initiative).
  - Методические рекомендации Минэкономразвития России для подготовки отчетности по 

In [18]:
len(res4)

1457

In [24]:
from PIL import Image
def create_texts_from_images(images_dir_path, output_dir):
    # Проверяем, существует ли папка для текстовых файлов, если нет, создаем её
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Проходим по каждому файлу в папке с изображениями
    for image_filename in os.listdir(input_dir):
        # Полный путь к изображению
        image_path = os.path.join(input_dir, image_filename)

        # Проверяем, что это изображение (например, файл с расширением PNG или JPG)
        if image_filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            # Открываем изображение, если оно действительно является изображением
            try:
                with Image.open(image_path):
                    result = prepare_text_from_image(image_path)

                    # Генерируем имя файла для текстового результата
                    output_filename = os.path.splitext(image_filename)[0] + '.txt'
                    output_path = os.path.join(output_dir, output_filename)

                    # Записываем результат функции в текстовый файл
                    with open(output_path, 'w', encoding='utf-8') as f:
                        f.write(result)
                    print(f'Результат для {image_filename} сохранен в {output_filename}')
            except Exception as e:
                print(f'Ошибка при обработке {image_filename}: {e}')

# Пример использования функции
input_dir = '../data/interim/images/'  # Укажи путь к папке с изображениями
# output_dir = os.path.join(input_dir, 'txt_files')  # Создаем новую папку рядом с исходной
output_dir = '../data/interim/texts/'

create_texts_from_images(input_dir, output_dir)

Результат для page_1.png сохранен в page_1.txt
Результат для page_10.png сохранен в page_10.txt
Результат для page_11.png сохранен в page_11.txt
Результат для page_12.png сохранен в page_12.txt
Результат для page_13.png сохранен в page_13.txt
Результат для page_14.png сохранен в page_14.txt
Результат для page_15.png сохранен в page_15.txt
Результат для page_16.png сохранен в page_16.txt
Результат для page_17.png сохранен в page_17.txt
Результат для page_18.png сохранен в page_18.txt
Результат для page_19.png сохранен в page_19.txt
Результат для page_2.png сохранен в page_2.txt
Результат для page_20.png сохранен в page_20.txt
Результат для page_3.png сохранен в page_3.txt
Результат для page_4.png сохранен в page_4.txt
Результат для page_5.png сохранен в page_5.txt
Результат для page_6.png сохранен в page_6.txt
Результат для page_7.png сохранен в page_7.txt
Результат для page_8.png сохранен в page_8.txt
Результат для page_9.png сохранен в page_9.txt
